<a href="https://colab.research.google.com/github/avirichie/Deep-Learning-Models/blob/master/My_Keras03_Multiclass_Classification(Reuters).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from tensorflow.keras.datasets import reuters
(X_train,Y_train),(X_test,Y_test) = reuters.load_data(num_words=10000)

In [0]:
X_train.shape

(8982,)

In [0]:
X_test.shape

(2246,)

You can’t feed lists of integers into a neural network. You have to turn your lists into tensors. There are two ways to do that:

Pad your lists so that they all have the same length, turn them into an integer tensor of shape (samples, word_indices), and then use as the first layer in your network a layer capable of handling such integer tensors.

One-hot encode your lists to turn them into vectors of 0s and 1s. This would mean, for instance, turning the sequence [3, 5] into a 10,000-dimensional vector that would be all 0s except for indices 3 and 5, which would be 1s. Then you could use as the first layer in your network a Dense layer, capable of handling floating-point vector data.

In [0]:
#Vectorize the Training data
import numpy as np 

def Vectorize(sequences,dimensions=10000):
  results=np.zeros((len(sequences),dimensions))
  for i,sequence in enumerate(sequences):
    results[i,sequence]=1
  return results

X_train_vec = Vectorize(X_train)
X_test_vec = Vectorize(X_test)

In [0]:
#Converting the list of integers into tensors
from tensorflow.keras.utils import to_categorical #Donot use np.utils here because its internal and cant be accessed publically, Thus import it just using utils and not utils.nputils

one_hot_Y_train =to_categorical(Y_train)
one_hot_Y_test  =to_categorical(Y_test)



In [0]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(layers.Dense(128,activation='relu',input_shape=(10000,)))
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dense(128,activation='relu'))
model.add(layers.Dense(46,activation='softmax')) # The last layer should have the 46 nodes because we have to classify the training example into 46 different categories

The best loss function to use in this case is categorical_crossentropy. It measures the distance between two probability distributions: here,
between the** probability distribution output by the network** and **the true distribution of the labels**. By minimizing the distance between these two distributions, you train the network to output something as close as possible to the true labels.

In [0]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
X_val = X_train_vec[:1000]
partial_X_train = X_train_vec[1000:]

Y_val = one_hot_Y_train[:1000]
partial_Y_train = one_hot_Y_train[1000:]

In [0]:
history = model.fit(x=X_train_vec,
                    y=one_hot_Y_train,
                    epochs=50,
                    batch_size=512,
                    validation_data=(X_val,Y_val))



```
The model after 50 epochs has a training accuracy of 95.81% and validation accuracy of 97.50%. 




In [53]:
final_results =  model.evaluate(X_test_vec,one_hot_Y_test)
print(final_results)

2246/2246 [==============================] - 0s 155us/sample - loss: 3.1988 - acc: 0.7707
[3.198813250737857, 0.7707035]


In [0]:
predictions = model.predict(X_test_vec)

In [55]:
predictions[0].shape

(46,)

In [58]:
np.argmax(predictions[0]) # The class with highest probability 

3